In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import alexnet
from tqdm import tqdm

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [13]:

# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224 for AlexNet
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=16),  # Adjust padding to fit the larger size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])  # CIFAR-10 normalization
])

transform_test = transforms.Compose([
    transforms.Resize(224),  # Resize test images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])  # CIFAR-10 normalization
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [14]:
# Load pre-defined AlexNet model and modify for CIFAR-10
model = alexnet(pretrained=False)
num_classes = 10
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
# Training function
def train(model, trainloader, testloader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(trainloader)):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Log training loss
        epoch_loss = running_loss / len(trainloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

        # Evaluate on the test set after each epoch
        test_loss, test_accuracy = evaluate(model, testloader)
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

# Evaluation function for validation loop
def evaluate(model, testloader):
    model.eval()
    total_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    total_loss /= len(testloader)
    accuracy = 100.0 * correct / len(testloader.dataset)
    return total_loss, accuracy



In [16]:
# Train the model for specified number of epochs
epochs = 50
train(model, trainloader, testloader, epochs)

# Save model checkpoint
torch.save(model.state_dict(), 'alexnet_cifar10.pth')
print("Model checkpoint saved.")


100%|██████████| 782/782 [00:51<00:00, 15.18it/s]

Epoch [1/50], Loss: 1.7563


Test Loss: 1.4225, Test Accuracy: 46.81%


100%|██████████| 782/782 [00:50<00:00, 15.33it/s]

Epoch [2/50], Loss: 1.4799


...

Epoch [49/50], Loss: 0.7926


Test Loss: 0.7499, Test Accuracy: 74.37%


100%|██████████| 782/782 [00:51<00:00, 15.27it/s]

Epoch [50/50], Loss: 0.8008


Test Loss: 0.7191, Test Accuracy: 76.13%
Model checkpoint saved.
